# 센서데이터 유저별 통합

In [ ]:
# 19년도 데이터 전처리 - User 101

import os
from time import strftime
from time import localtime
import pandas as pd
from tqdm import tqdm

##########################
# user 101 데이터에서 센서별 파일의 제목을 epoch time에서 실제 시간으로 변경
sensor = ['mAcc', 'mGps', 'mGyr', 'mMag', 'e4Acc', 'e4Bvp', 'e4Eda', 'e4Hr', 'e4Temp']
time = os.listdir('D:/애트리/dataset_2019/101/')

for sen in sensor:
    for ti in time:
        di = 'D:/애트리/dataset_2019/101/' + ti + '/' + sen

        for num in os.listdir(di):
            os.rename(di + '/' + num, di + '/' + strftime('%Y_%m_%d_%I_%M_%S_%p', localtime(int(num[:-4]))) + '.csv')
########################


# 센서 파일이 기록된 시간 단위로 무슨 행동을 하고 있었는지, 어디에 있었는지 등등
# 상세 label이 기록되어 있는 파일을 하나의 DataFrame으로 합치는 코드
df_total = pd.DataFrame({'ts':[], 'action':[], 'actionOption':[], 'actionSub':[], 'actionSubOption':[],
       'condition':[], 'conditionSub1Option':[], 'conditionSub2Option':[], 'place':[],
       'emotionPositive':[], 'emotionTension':[], 'activity':[]})

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/' + d + '_label.csv'
    
    df = pd.read_csv(di_a)
    df_total = pd.concat([df_total, df])

# epoch time을 실제 시간으로 변환
ta = []
for i in df_total['ts']:
    ta.append(strftime('%Y_%m_%d_%I_%M_%S_%p', localtime(int(i))))
df_total['ts'] = ta

#######################
# 센서별로 데이터 파일 1분단 위로 저장 -> 각각의 파일 속에도 초 단위로 센서 기록. 
# 분 단위로 기록된 센서의 평균 및 표준편차를 저장하는 코드
# mAcc (휴대폰 가속도 센서)

di = 'D:/애트리/dataset_2019/101/'

x_mean = []
y_mean = []
z_mean = []
x_std = []
y_std = []
z_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/mAcc/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        x_mean.append(df['x'].mean())
        y_mean.append(df['y'].mean())
        z_mean.append(df['z'].mean())
        x_std.append(df['x'].std())
        y_std.append(df['y'].std())
        z_std.append(df['z'].std())
        time.append(file[:-4])

# label 데이터와 센서 데이터(macc)를 병합
macc = pd.DataFrame({'ts':time, 'macc_x_mean':x_mean, 'macc_x_std':x_std, 'macc_y_mean':y_mean, 'macc_y_std':y_std, 'macc_z_mean':z_mean, 'macc_z_std':z_std})
df_total = pd.merge(df_total, macc, on='ts', how='left')


# mgps(휴대폰 GPS 센서)
lat_mean = []
lat_std = []
lon_mean = []
lon_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/mGps/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        lat_mean.append(df['lat'].mean())
        lon_mean.append(df['lon'].mean())
        lat_std.append(df['lat'].std())
        lon_std.append(df['lon'].std())
        time.append(file[:-4])
                    
mgps = pd.DataFrame({'ts':time, 'mgps_lat_mean':lat_mean, 'mgps_lat_std':lat_std, 'mgps_lon_mean':lon_mean, 'mgps_lon_std':lon_std})
df_total = pd.merge(df_total, mgps, on='ts', how='left')


# e4Acc(스마트워치 가속도 센서)
x_mean = []
y_mean = []
z_mean = []
x_std = []
y_std = []
z_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/e4Acc/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        x_mean.append(df['x'].mean())
        y_mean.append(df['y'].mean())
        z_mean.append(df['z'].mean())
        x_std.append(df['x'].std())
        y_std.append(df['y'].std())
        z_std.append(df['z'].std())
        time.append(file[:-4])
        

e4acc = pd.DataFrame({'ts':time, 'e4acc_x_mean':x_mean, 'e4acc_x_std':x_std, 'e4acc_y_mean':y_mean, 'e4acc_y_std':y_std, 'e4acc_z_mean':z_mean, 'e4acc_z_std':z_std})
df_total = pd.merge(df_total, e4acc, on='ts', how='left')


# e4Bvp(스마트워치 혈압 데이터)
bvp_mean = []
bvp_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/e4Bvp/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        time.append(file[:-4])
        bvp_mean.append(df['x'].mean())
        bvp_std.append(df['x'].std())
        

e4bvp = pd.DataFrame({'ts':time, 'bvp_mean':bvp_mean, 'bvp_std':bvp_std})
df_total = pd.merge(df_total, e4bvp, on='ts', how='left')


# e4Hr(스마트워치 심박수 데이터)
hr_mean = []
hr_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/e4Hr/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        time.append(file[:-4])
        hr_mean.append(df['x'].mean())
        hr_std.append(df['x'].std())

e4hr = pd.DataFrame({'ts':time, 'hr_mean':hr_mean, 'hr_std':hr_std})
df_total = pd.merge(df_total, e4hr, on='ts', how='left')


# e4Temp(스마트워치 체온 데이터)
temp_mean = []
temp_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/e4Temp/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        time.append(file[:-4])
        temp_mean.append(df['x'].mean())
        temp_std.append(df['x'].std())
        
e4temp = pd.DataFrame({'ts':time, 'temp_mean':temp_mean, 'temp_std':temp_std})
df_total = pd.merge(df_total, e4temp, on='ts', how='left')

df_total.to_csv('D:/애트리/dataset_2019/df_total_101.csv', index=False)

In [ ]:
# 20년도 데이터 전처리 - user01
# 과정은 위 코드와 동일, csv 파일 컬럼명만 19년과 살짝 다름

sensor = ['mAcc', 'mGps', 'mGyr', 'mMag', 'e4Acc', 'e4Bvp', 'e4Eda', 'e4Hr', 'e4Temp']
time = os.listdir('D:/애트리/user01-06/user01/')

for sen in sensor:
    for ti in time:
        di = 'D:/애트리/user01-06/user01/' + ti + '/' + sen

        for num in os.listdir(di):
            os.rename(di + '/' + num, di + '/' + strftime('%Y_%m_%d_%I_%M_%S_%p', localtime(int(num[:-4]))) + '.csv')

            
di = 'D:/애트리/user01-06/user01/'

df_total = pd.DataFrame({'ts':[], 'action':[], 'actionOption':[], 'actionSub':[], 'actionSubOption':[],
       'condition':[], 'conditionSub1Option':[], 'conditionSub2Option':[], 'place':[],
       'emotionPositive':[], 'emotionTension':[], 'activity':[]})

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/' + d + '_label.csv'
    
    df = pd.read_csv(di_a)
    df_total = pd.concat([df_total, df])


ta = []
for i in df_total['ts']:
    ta.append(strftime('%Y_%m_%d_%I_%M_%S_%p', localtime(int(i))))

df_total['ts'] = ta


x_mean = []
y_mean = []
z_mean = []
x_std = []
y_std = []
z_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/mAcc/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        x_mean.append(df['x'].mean())
        y_mean.append(df['y'].mean())
        z_mean.append(df['z'].mean())
        x_std.append(df['x'].std())
        y_std.append(df['y'].std())
        z_std.append(df['z'].std())
        time.append(file[:-4])


macc = pd.DataFrame({'ts':time, 'macc_x_mean':x_mean, 'macc_x_std':x_std, 'macc_y_mean':y_mean, 'macc_y_std':y_std, 'macc_z_mean':z_mean, 'macc_z_std':z_std})
df_total = pd.merge(df_total, macc, on='ts', how='left')


lat_mean = []
lat_std = []
lon_mean = []
lon_std = []
time = []
from tqdm import tqdm
for d in tqdm(os.listdir(di)):
    di_a = di + d + '/mGps/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        lat_mean.append(df['lat'].mean())
        lon_mean.append(df['lon'].mean())
        lat_std.append(df['lat'].std())
        lon_std.append(df['lon'].std())
        time.append(file[:-4])
            
mgps = pd.DataFrame({'ts':time, 'mgps_lat_mean':lat_mean, 'mgps_lat_std':lat_std, 'mgps_lon_mean':lon_mean, 'mgps_lon_std':lon_std})
df_total = pd.merge(df_total, mgps, on='ts', how='left')


x_mean = []
y_mean = []
z_mean = []
x_std = []
y_std = []
z_std = []
time = []


from tqdm import tqdm
for d in tqdm(os.listdir(di)):
    di_a = di + d + '/e4Acc/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        x_mean.append(df['x'].mean())
        y_mean.append(df['y'].mean())
        z_mean.append(df['z'].mean())
        x_std.append(df['x'].std())
        y_std.append(df['y'].std())
        z_std.append(df['z'].std())
        time.append(file[:-4])

e4acc = pd.DataFrame({'ts':time, 'e4acc_x_mean':x_mean, 'e4acc_x_std':x_std, 'e4acc_y_mean':y_mean, 'e4acc_y_std':y_std, 'e4acc_z_mean':z_mean, 'e4acc_z_std':z_std})
df_total = pd.merge(df_total, e4acc, on='ts', how='left')



bvp_mean = []
bvp_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/e4Bvp/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        time.append(file[:-4])
        bvp_mean.append(df['value'].mean())
        bvp_std.append(df['value'].std())

e4bvp = pd.DataFrame({'ts':time, 'bvp_mean':bvp_mean, 'bvp_std':bvp_std})
df_total = pd.merge(df_total, e4bvp, on='ts', how='left')



hr_mean = []
hr_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/e4Hr/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        time.append(file[:-4])
        hr_mean.append(df['hr'].mean())
        hr_std.append(df['hr'].std())
        
e4hr = pd.DataFrame({'ts':time, 'hr_mean':hr_mean, 'hr_std':hr_std})
df_total = pd.merge(df_total, e4hr, on='ts', how='left')


temp_mean = []
temp_std = []
time = []

for d in tqdm(os.listdir(di)):
    di_a = di + d + '/e4Temp/'
    
    for file in os.listdir(di_a):
        df = pd.read_csv(di_a + file)
        time.append(file[:-4])
        temp_mean.append(df['temp'].mean())
        temp_std.append(df['temp'].std())
        

e4temp = pd.DataFrame({'ts':time, 'temp_mean':temp_mean, 'temp_std':temp_std})

df_total = pd.merge(df_total, e4temp, on='ts', how='left')
df_total.to_csv('D:/애트리/user11-12/df_total_01.csv', index=False)

# 유저별 데이터 통합(20년도)

In [3]:
import pandas as pd
import os

# 유저별로 데이터를 통합해주기 위해 컬럼만 있는 빈 데이터프레임을 생성
col_ls = list(pd.read_csv('D:/애트리/2020_data/df_total_01.csv').columns)
col_ls.append('ts_group')
df = pd.DataFrame({i : [] for i in col_ls})
df

,ts,action,actionOption,actionSub,actionSubOption,condition,conditionSub1Option,conditionSub2Option,place,emotionPositive,...,e4acc_y_std,e4acc_z_mean,e4acc_z_std,bvp_mean,bvp_std,hr_mean,hr_std,temp_mean,temp_std,ts_group


In [4]:
# 2020_data 폴더에 있는 유저별 통합 파일을 하나씩 읽어들여서 df에 통합
# ts_group 컬럼은 ts 값은 똑같지만 user가 다른 경우가 있기 때문에 이를 구분해주기 위해 추가

for i in os.listdir('D:/애트리/2020_data'):
    df_a = pd.read_csv('D:/애트리/2020_data/' + i)
    ts_group = []
    df_a['ts'] = pd.to_datetime(df_a['ts'], format='%Y_%m_%d_%I_%M_%S_%p')
    for j in range(df_a.shape[0]):
        ts_group.append(df_a['ts'].iloc[j].strftime("%Y-%m-%d") + '_' + i[-6:-4])
        
    df_a['ts_group'] = ts_group
    df = pd.concat([df, df_a], axis=0)

In [6]:
df

,ts,action,actionOption,actionSub,actionSubOption,condition,conditionSub1Option,conditionSub2Option,place,emotionPositive,...,e4acc_y_std,e4acc_z_mean,e4acc_z_std,bvp_mean,bvp_std,hr_mean,hr_std,temp_mean,temp_std,ts_group
0,2020-08-30 12:58:00,work,212.0,NaN,NaN,WITH_MANY,4.0,3.0,other_indoor,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-30_01
1,2020-08-30 12:59:00,work,212.0,NaN,NaN,WITH_MANY,4.0,3.0,other_indoor,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-30_01
2,2020-08-30 13:00:00,work,212.0,NaN,NaN,WITH_MANY,4.0,3.0,other_indoor,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-30_01
3,2020-08-30 13:01:00,work,212.0,NaN,NaN,WITH_MANY,4.0,3.0,other_indoor,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-30_01
4,2020-08-30 13:02:00,work,212.0,NaN,NaN,WITH_MANY,4.0,3.0,other_indoor,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-30_01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22266,2020-09-27 22:45:00,recreation_media,725.0,NaN,NaN,ALONE,NaN,NaN,home,7.0,...,0.356393,0.659635,0.292375,0.321894,73.025768,74.283333,2.629190,32.435667,0.024842,2020-09-27_30
22267,2020-09-27 22:46:00,recreation_media,725.0,NaN,NaN,ALONE,NaN,NaN,home,7.0,...,0.325196,0.812223,0.263424,0.120625,47.997696,79.778667,2.140893,32.551000,0.041183,2020-09-27_30
22268,2020-09-27 22:47:00,recreation_media,725.0,NaN,NaN,ALONE,NaN,NaN,home,7.0,...,0.093206,0.500260,0.385400,-0.235474,61.118549,74.824167,1.075296,32.587667,0.017583,2020-09-27_30
22269,2020-09-27 22:48:00,recreation_media,725.0,NaN,NaN,ALONE,NaN,NaN,home,7.0,...,0.062955,0.665975,0.029459,-0.107089,56.698782,67.195167,1.441878,32.578000,0.012781,2020-09-27_30


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440830 entries, 0 to 22270
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ts                   440830 non-null  object 
 1   action               440830 non-null  object 
 2   actionOption         440830 non-null  float64
 3   actionSub            95860 non-null   object 
 4   actionSubOption      95860 non-null   float64
 5   condition            440830 non-null  object 
 6   conditionSub1Option  207576 non-null  float64
 7   conditionSub2Option  207576 non-null  float64
 8   place                440830 non-null  object 
 9   emotionPositive      440830 non-null  float64
 10  emotionTension       440830 non-null  float64
 11  activity             440830 non-null  float64
 12  macc_x_mean          405014 non-null  float64
 13  macc_x_std           405014 non-null  float64
 14  macc_y_mean          405014 non-null  float64
 15  macc_y_std        

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# 센서데이터 중 결측치를 제거
df = df.dropna(subset=['macc_x_mean', 'mgps_lat_std', 'e4acc_x_mean', 'temp_mean', 'bvp_std', 'hr_std'], axis=0).copy()

# 활동이 sleep, recreation_etc, meal, hobby인 경우를 제거
# sleep : 다른 파일에 존재 (user_sleep_2020.csv)
# recreation etc, hobby : 행동의 범주가 너무 다양해서 정상적인 예측 어려움
# meal : 식사 또한 추정 어려움
df = df[(df['action']!='sleep')&(df['action']!='recreation_etc')&(df['action']!='meal')&(df['action']!='hobby')]

# action 컬럼을 예측하기 위해 필요한 변수를 구분하여 저장
X = df.drop(['ts', 'action', 'actionOption', 'actionSub', 'actionSubOption', 'condition', 'conditionSub1Option', 'conditionSub2Option', 'place', 'emotionPositive', 'emotionTension', 'activity', 'ts_group'], axis=1)
y = df['action']

# test 데이터 비율 0.25로 train, test 셋을 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# 센서데이터 특성상 이상치가 어느정도 존재할 수 밖에 없기 때문에, 여러 Scaler 중 이상치에 가장 강인한 RobustScaler를 사용
for col in X_train.columns:
    scal = RobustScaler()
    X_train[col] = scal.fit_transform(np.array(X_train[col]).reshape(-1, 1))
    X_test[col] = scal.transform(np.array(X_test[col]).reshape(-1, 1))

classifier = RandomForestClassifier(n_estimators = 300)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print("정확도 : {}".format(accuracy_score(y_test, y_pred)))

정확도 : 0.855109762748841


In [9]:
# 실제 상황에서는 센서 데이터를 기반으로 행동을 추정하므로(label 값이 없음)
# 예측한 활동을 새로운 컬럼에 저장(이를 실제 활동이라고 여기고 사용할 것임)

pred_class = classifier.predict(X)
pred_class.shape

(234687,)

In [10]:
df['pred'] = pred_class

# 불필요한 컬럼을 제거
df = df.drop(['action', 'actionOption', 'actionSub', 'actionSubOption', 'condition', 'conditionSub1Option', 'conditionSub2Option', 'place', 'emotionPositive', 'emotionTension', 'activity'], axis=1)

In [13]:
df.to_csv('D:/애트리/df_total_pred.csv', index=False)

In [14]:
df = pd.read_csv('D:/애트리/df_total_pred.csv')

# time step을 datetime 형식으로 변경
df['ts'] = pd.to_datetime(df['ts'])

In [15]:
# 2020년도 유저들의 action별 활동시간을 기록하기 위해, 모든 action을 컬럼으로 생성
# ts_group은 앞에서와 마찬가지로 user별로 time step이 중복되기 때문에 이를 구분하기 위해 추가

ls_ac = list(df['pred'].unique())
ls_ac.append('ts_group')
dic_total = {i : 0 for i in ls_ac}
dic_total

{'care_housemem': 0,
 'household': 0,
 'travel': 0,
 'recreation_media': 0,
 'outdoor_act': 0,
 'ts_group': 0}

In [16]:
# 하루 당 유저별 활동량(예측 활동) 계산

df_time = pd.DataFrame({i:[] for i in ls_ac})
from tqdm import tqdm

for i in tqdm(df['ts_group'].unique()):
    df_sample = df[df['ts_group'] == i]
    
    # 활동 시간을 계산하는데 ts_group은 필요하지 않기 때문에 마지막 값 제거
    dic_total = {i : 0 for i in ls_ac[:-1]}
    
    for j in range(df_sample.shape[0]-1):
        # t+1기의 time step의 시간 - t기의 time step의 시간 차이로 활동 시간을 계산
        td = df_sample['ts'].iloc[j+1] - df_sample['ts'].iloc[j]
        col = df_sample.iloc[j]['pred']
        
        # 활동시간을 t기 timestep의 활동 분야로 기록
        dic_total[col] += td.total_seconds()/60
        
    df_app = pd.DataFrame(dic_total, index=[0])
    df_app['ts_group'] = i
        
    df_time = pd.concat([df_time, df_app], axis=0, ignore_index=True)


100%|████████████████████████████████████████████████████████████████████████████████| 482/482 [00:53<00:00,  9.09it/s]


In [17]:
df_time

,care_housemem,household,travel,recreation_media,outdoor_act,ts_group
0,767.0,10.0,0.0,0.0,0.0,2020-08-31_01
1,734.0,17.0,0.0,0.0,0.0,2020-09-05_01
2,739.0,8.0,4.0,0.0,0.0,2020-09-07_01
3,611.0,15.0,4.0,0.0,0.0,2020-09-08_01
4,42.0,3.0,1.0,0.0,0.0,2020-09-09_01
...,...,...,...,...,...,...
477,893.0,2.0,5.0,3.0,0.0,2020-09-23_30
478,793.0,1.0,0.0,0.0,0.0,2020-09-24_30
479,762.0,1.0,11.0,3.0,0.0,2020-09-25_30
480,769.0,0.0,0.0,0.0,0.0,2020-09-26_30


In [18]:
# 수면 점수와의 연관성을 위해 수면 데이터를 읽어들임

import pandas as pd
sleep = pd.read_csv('D:/애트리/user11-12/user_sleep_2020.csv')

In [19]:
# user 별 time step을 구분하기 위해 sleep에도 추가
ts_group = []

for i in range(sleep.shape[0]):
    ts_group.append(sleep['date'].iloc[i] + '_' + sleep['userId'].iloc[i][4:])

sleep['ts_group'] = ts_group

In [20]:
sleep = sleep[['lightsleepduration', 'deepsleepduration', 'remsleepduration', 'sleep_score', 'ts_group']]
sleep

,lightsleepduration,deepsleepduration,remsleepduration,sleep_score,ts_group
0,13800,9000,5280,86,2020-08-30_24
1,9540,6360,4980,52,2020-08-31_24
2,8760,4740,7440,61,2020-09-01_24
3,7740,7440,2220,40,2020-09-02_24
4,11640,7620,2040,55,2020-09-03_24
...,...,...,...,...,...
610,11820,4560,5940,60,2020-10-05_06
611,13200,7500,4680,79,2020-10-06_06
612,8520,1500,0,20,2020-10-07_06
613,9240,4560,1080,20,2020-10-08_06


In [21]:
# 낮잠을 자는 등의 변수를 제거하기 위해 하루에 2번 이상 잔 데이터는 제거

a = sleep['ts_group'].value_counts()[sleep['ts_group'].value_counts() == 1]
rm_ls = []

for i in range(sleep.shape[0]):
    if sleep['ts_group'].iloc[i] in a.index:
        rm_ls.append(i)

In [22]:
sleep = sleep.iloc[rm_ls]

In [23]:
df_final = pd.merge(df_time, sleep, how='inner', on='ts_group')
df_final

,care_housemem,household,travel,recreation_media,outdoor_act,ts_group,lightsleepduration,deepsleepduration,remsleepduration,sleep_score
0,767.0,10.0,0.0,0.0,0.0,2020-08-31_01,11700,6120,7500,83
1,734.0,17.0,0.0,0.0,0.0,2020-09-05_01,12480,7140,5760,79
2,611.0,15.0,4.0,0.0,0.0,2020-09-08_01,16980,7080,4860,83
3,42.0,3.0,1.0,0.0,0.0,2020-09-09_01,8700,5760,4380,51
4,490.0,28.0,0.0,0.0,0.0,2020-09-12_01,12120,10500,5340,93
...,...,...,...,...,...,...,...,...,...,...
378,861.0,1.0,3.0,1.0,0.0,2020-09-21_30,12660,2400,7620,57
379,738.0,1.0,13.0,2.0,0.0,2020-09-22_30,11340,4200,6420,56
380,893.0,2.0,5.0,3.0,0.0,2020-09-23_30,10200,3240,10140,74
381,762.0,1.0,11.0,3.0,0.0,2020-09-25_30,8820,4320,7680,56


In [24]:
df_final.to_csv('D:/애트리/df_final.csv', index=False)